In [1]:
import transformers
import huggingface_hub
from huggingface_hub import notebook_login
import ipywidgets

In [2]:
notebook_login()

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel, Trainer, TrainingArguments
import pandas as pd 
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("data/tweet_text_2.csv", encoding = 'cp1252') 
tweet_text = data['TWEET_TEXT']
tweet_lengths = [len(tweet) for tweet in tweet_text]


Finetuning BERT to see performance for predicting gasoline shortage

In [19]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding=max, truncation=True)
model = AutoModel.from_pretrained(model_name)

tokenizing data and creating trainign an test sets 

In [ ]:
data["tokenized_tweets"] = data['TWEET_TEXT'].apply(lambda x: tokenizer(x))
train_data = data.sample(frac =0.8)
test_data = data[~data.index.isin(train_data)]

creating a the model to finetune

In [30]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
class Model(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.bert = model
        self.dropout = nn.Dropout(0.2)
        self.linear = nn.Linear(768,2)
        self.softmax = nn.Softmax(1)
    def forward(self, x):
        bert_output = model(x)
        output = self.softmax(self.linear(bert_output))
        return output
        

finetune_model = Model()

False